<a href="https://colab.research.google.com/github/MATHKIMS/dongwoo_project/blob/main/%EB%AF%B8%EB%8B%88%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B85_ResNet_34.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


지도학습:CNN 

In [2]:
#코랩에서 GPU 사용
import torch
#=== GPU 사용하는지 확인===#

USE_GPU= torch.cuda.is_available()
print(USE_GPU)

#디바이스 확인
DEVICE=torch.device("cuda" if USE_GPU else "cpu")
print(f"사용 중인 DEVICE : {DEVICE}")
#=====================================================#

True
사용 중인 DEVICE : cuda


In [3]:
import zipfile

z_file=zipfile.ZipFile('/content/drive/MyDrive/class_10.zip','r') #r은 읽기형식으로
name_list=z_file.namelist()
print(type(name_list))
print(name_list)

name_list=[name for name in name_list if name[-1]!='/']
cls=[int(cls.split('/')[0]) for cls in name_list]
print(name_list)
print(cls)
print(len(name_list),len(cls))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(name_list, cls, test_size=0.2, random_state=42, stratify=cls)
print(len(X_train),len(y_train))
print(len(X_test),len(y_test))

<class 'list'>
['0/', '0/0.JPEG', '0/1.JPEG', '0/10.JPEG', '0/100.JPEG', '0/1000.JPEG', '0/1001.JPEG', '0/1002.JPEG', '0/1003.JPEG', '0/1004.JPEG', '0/1005.JPEG', '0/1006.JPEG', '0/1007.JPEG', '0/1008.JPEG', '0/1009.JPEG', '0/101.JPEG', '0/1010.JPEG', '0/1011.JPEG', '0/1012.JPEG', '0/1013.JPEG', '0/1014.JPEG', '0/1015.JPEG', '0/1016.JPEG', '0/1017.JPEG', '0/1018.JPEG', '0/1019.JPEG', '0/102.JPEG', '0/1020.JPEG', '0/1021.JPEG', '0/1022.JPEG', '0/1023.JPEG', '0/1024.JPEG', '0/1025.JPEG', '0/1026.JPEG', '0/1027.JPEG', '0/1028.JPEG', '0/1029.JPEG', '0/103.JPEG', '0/1030.JPEG', '0/1031.JPEG', '0/1032.JPEG', '0/1033.JPEG', '0/1034.JPEG', '0/1035.JPEG', '0/1036.JPEG', '0/1037.JPEG', '0/1038.JPEG', '0/1039.JPEG', '0/104.JPEG', '0/1040.JPEG', '0/1041.JPEG', '0/1042.JPEG', '0/1043.JPEG', '0/1044.JPEG', '0/1045.JPEG', '0/1046.JPEG', '0/1047.JPEG', '0/1048.JPEG', '0/1049.JPEG', '0/105.JPEG', '0/1050.JPEG', '0/1051.JPEG', '0/1052.JPEG', '0/1053.JPEG', '0/1054.JPEG', '0/1055.JPEG', '0/1056.JPEG', '0

In [4]:
import numpy as np
import cv2
def load_train_batch(z_file, batch_size, x_name,y, n_class=10):
  x_batch=np.zeros((batch_size,96,96,3))
  y_batch=np.zeros((batch_size,n_class))
  rand_idx=np.random.randint(0,len(x_name),batch_size)
  count=0

  for idx in rand_idx:
    img=z_file.read(x_name[idx])
    img=np.frombuffer(img,dtype=np.int8)
    img=cv2.imdecode(img,cv2.IMREAD_COLOR)
    h_rand,w_rand=np.random.randint(0,33,2)
    img=img[h_rand:h_rand+96, w_rand:w_rand+96,:]
   #원핫인코딩 정답레이블 만들기
    img =(img-np.mean(img,axis=(0,1)))/np.std(img,axis=(0,1))
    x_batch[count]=img
    y_batch[count,y[idx]]= 1  
    count += 1
  x_batch=torch.FloatTensor(x_batch) #텐서로 바꾼것 의도: 그래디언트 3D 차원으로 바꿔놓은것
  y_batch=torch.FloatTensor(y_batch)
  x_batch=x_batch.permute(0,3,1,2) # permute?

  return x_batch,y_batch

### 네트워크 설계 (심플)

In [5]:
from torch.nn.modules import ReLU
from torch.nn.modules.batchnorm import BatchNorm2d


import torch
import torch.nn as nn
import torch.nn.functional as F

class Simple_CNN(nn.Module): #클래스 상속
  def __init__(self):
    super(Simple_CNN,self).__init__() 
    
    self.header = nn.Sequential(
        nn.Conv2d(3,64,kernel_size=7,stride=2,padding=4), #nn.Conv2d(입력채널,출력채널,커널사이즈
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(3,stride=2)
    ) #Sequential 순차적 뉴럴네트워크 구성 실행

    self.block_1=nn.Sequential(
        nn.Conv2d(64,64,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64,64,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64,64,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64,64,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(2,stride=2)
    )

    self.block_2=nn.Sequential(
        nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Conv2d(128,128,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Conv2d(128,128,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Conv2d(128,128,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(2,stride=2)
    )

    self.block_3=nn.Sequential(
        nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(2,stride=2)
    )

    self.block_4=nn.Sequential(
        nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(2,stride=2)
    )

    self.linear=nn.Sequential(
          nn.Linear(512,10)
    )
  def forward(self,input):
    x= self.header(input)
    x= self.block_1(x)
    x= self.block_2(x)
    x= self.block_3(x)
    x= self.block_4(x)

    x=F.avg_pool2d(x,kernel_size=x.shape[2]) #x.shape[2]=x_width
    x=torch.flatten(x,start_dim=1)
    x= self.linear(x)
    return x 

#(Batch,Demension,Height,Width)


In [6]:
model=Simple_CNN()

In [7]:
def train(model,optimizer,z_file,X_train,y_train,batch_size,n_class,DEVICE,epoch):
  model.train()
  for i in range(len(X_test)//batch_size):
    data,target = load_train_batch(z_file,batch_size,X_train,y_train,n_class)
    data,target = data.to(DEVICE),target.to(DEVICE)
    optimizer.zero_grad()
    y_pred=model(data)
    criterion=nn.CrossEntropyLoss()
    loss=criterion(y_pred,target)
    loss.backward()
    optimizer.step()
    if i%100==0:
      print(f"Train [{epoch}] [{i}]/{len(X_train)//batch_size} Loss:{loss.item():.6f}")

In [8]:
def test(model,z_file,X_test,y_test,batch_size,n_class,DEVICE,epoch,mode):
  model.eval()
  total_count=len(X_test)
  correct_count=0
  with torch.no_grad():
    while(len(X_test))!=0:
      if len(X_test)<batch_size:
        batch_size=len(X_test)


      x_batch=np.zeros((batch_size,128,128,3)) #원본이미지 테스트를 위한 배치사이즈 지정 
      y_batch=np.zeros(batch_size)
    
      for idx in range(batch_size):
        img=z_file.read(X_test.pop())
        img=np.frombuffer(img,np.int8)
        img=cv2.imdecode(img, cv2.IMREAD_COLOR)
        img=(img-np.mean(img,axis=(0,1)))/np.std(img,axis=(0,1))
        
        cls=y_test.pop()

        x_batch[idx] = img
        y_batch[idx] = cls
      x_batch=torch.FloatTensor(x_batch)
      y_batch=torch.FloatTensor(y_batch)
      x_batch=x_batch.permute(0,3,1,2)
      x_batch,y_batch=x_batch.to(DEVICE),y_batch.to(DEVICE)

      y_pred= model(x_batch)
      y_pred= torch.argmax(y_pred,dim=1)
      correct_count += torch.sum(y_pred == y_batch)
  
  print(f"{mode} [{epoch}] Accuracy:{correct_count/ total_count*100}")
  f=open('accuracy.txt','a+')
  f.write(f"{mode} [{epoch}] Accuracy:{correct_count/ total_count*100}\n")
  f.close()
  return

In [9]:
import torch.optim as optim

learning_rate=0.01
batch_size=32
max_epoch=50
n_class=10

model=Simple_CNN()
model=model.to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-4)
#오버피팅 검증을 위한 과정
for i in range(1,max_epoch+1):
  train(model,optimizer,z_file,X_train,y_train,batch_size,n_class,DEVICE,i)
  test(model,z_file,X_train.copy(),y_train.copy(),batch_size,n_class,DEVICE,i,'Train')
  test(model,z_file,X_test.copy(),y_test.copy(),batch_size,n_class,DEVICE,i,'Test')

Train [1] [0]/336 Loss:2.503166
Train [1] Accuracy:20.254457473754883
Test [1] Accuracy:20.72808265686035
Train [2] [0]/336 Loss:2.218481
Train [2] Accuracy:20.217309951782227
Test [2] Accuracy:20.72808265686035
Train [3] [0]/336 Loss:2.352142
Train [3] Accuracy:26.550891876220703
Test [3] Accuracy:26.300148010253906
Train [4] [0]/336 Loss:1.986459
Train [4] Accuracy:26.374441146850586
Test [4] Accuracy:25.965824127197266
Train [5] [0]/336 Loss:2.184945
Train [5] Accuracy:25.362184524536133
Test [5] Accuracy:26.040119171142578
Train [6] [0]/336 Loss:1.898271
Train [6] Accuracy:26.773775100708008
Test [6] Accuracy:26.114412307739258
Train [7] [0]/336 Loss:1.960489
Train [7] Accuracy:28.6125545501709
Test [7] Accuracy:28.41753387451172
Train [8] [0]/336 Loss:1.920838
Train [8] Accuracy:28.742568969726562
Test [8] Accuracy:28.083208084106445
Train [9] [0]/336 Loss:1.789660
Train [9] Accuracy:33.19093704223633
Test [9] Accuracy:32.689449310302734
Train [10] [0]/336 Loss:1.905890
Train [10]

그래디언트 소실문제 해결 모델

In [10]:
# 기존의 conv를 쌓았을 때 증폭과 소실에 대한 문제를 해결하기 위한 방법 (그레디언트 증폭과 소실.[어제 증폭이슈는 해결했고, 다음에서 소실에 대한 문제를 완화])

class Bottle_Neck(nn.Module):
  def __init__(self, input_channel, output_channel, is_downsample = False):
    super(Bottle_Neck, self).__init__()

    # Resnet 후속 논문을 보니 다음과 같은 방법의 효과가 가장 좋았다고 한다. ( ReLU를 거칠때 소실이 많이 일어나나봄 -> 주관적인 이야기 / 왜 ReLU를 안해주는지는 모름 다만 성능이 좋았음 )
    self.bottle_neck = nn.Sequential(
        nn.BatchNorm2d(input_channel),
        nn.ReLU(),
        nn.Conv2d(input_channel,output_channel,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(output_channel),
        nn.ReLU(),
        nn.Conv2d(output_channel,output_channel,kernel_size=3,stride=1,padding=1)
    )

    if is_downsample == True:  
      
      self.down_sample_conv = nn.Conv2d(input_channel, output_channel, kernel_size = 1,
                                       stride = 1, padding = 0)

  def forward(self, input):
    # indentity = input
    x = self.bottle_neck(input)
    if x.shape != input.shape:
      input = self.down_sample_conv(input) # -> 1X1 을 할땐 편향을 꺼주어야 한다?

    return x + input

In [11]:
class Res_net_18(nn.Module):
  def __init__(self):
    super(Res_net_18, self).__init__()

    self.header = nn.Sequential(
        nn.Conv2d(3,64, kernel_size = 7, stride = 2, padding = 4),
        # nn.Conv2d(입력 채널, 출력 채널, kernel_size = 커널 사이즈, stride = 보폭)
        nn.BatchNorm2d(64),
        # nn.BatchNorm2d(입력 채널) -> ReLU를 쓸꺼라서 노멀리제이션 해줌
        nn.ReLU(),
        nn.MaxPool2d(3, stride = 2)
    )

    # Sequential 순차적 뉴럴네트워크 구성 실행
    self.block_1 = nn.Sequential(
        Bottle_Neck(64,64),
        Bottle_Neck(64,64),
        nn.MaxPool2d(2, stride=2)
    )

    # 다음 블럭을 넘어갈때 인풋 채널과 아웃풋 채널이 달라 더할 수 없음.
    # Bottle_Neck 에 if is_downsample = True: 조건을 줌.

    self.block_2 = nn.Sequential(
        Bottle_Neck(64,128,is_downsample = True),
        Bottle_Neck(128,128),
        nn.MaxPool2d(2, stride=2)
    )

    self.block_3 = nn.Sequential(
        Bottle_Neck(128,256,is_downsample = True),
        Bottle_Neck(256,256),
        nn.MaxPool2d(2, stride=2)
    )

    self.block_4 = nn.Sequential(
        Bottle_Neck(256,512,is_downsample = True),
        Bottle_Neck(512,512),
        nn.MaxPool2d(2, stride=2),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )

    self.linear = nn.Sequential(
        nn.Linear(512, 10)
    )

  def forward(self, input):
    x = self.header(input)
    x = self.block_1(x)
    x = self.block_2(x)
    x = self.block_3(x)
    x = self.block_4(x)
    # print(x.shape)
    x = F.avg_pool2d(x, kernel_size = x.shape[2]) # global averge pooling
    # 커널사이즈를 마지막 W와 같게 해줌 -> 결과적으로 (32,512,3,3) or (32,512,7,7) -> (32,512,1,1)로 바뀌고
    x = torch.flatten(x, start_dim = 1)
    # flatten 해주면 (32,512)의 모양이 됨.
    x = self.linear(x)
    return x

In [12]:
def train(model, optimizer, z_file, x_train, y_train, batch_size, n_class, DEVICE, epoch):
  model.train()
  for i in range(len(x_train)//batch_size):
    data, target = load_train_batch(z_file, batch_size, x_train, y_train, n_class)
    data, target = data.to(DEVICE), target.to(DEVICE) # 맨~ 처음에 디바이스를 GPU로 할지 CPU로 할지 정해뒀기 때문에 각 위치에 데이터를 보내는것
    optimizer.zero_grad()
    y_pred = model(data)
    criterion = nn.CrossEntropyLoss() 
    loss = criterion(y_pred, target) # -> CrossEntropyLoss 함수가 y_pred에 softmax를 적용 후 target과의 loss를 구해줌
    loss.backward()
    optimizer.step()
    if i%100==0:
      print(f"Train [{epoch}] [{i}]/{len(x_train)//batch_size} Loss: {loss.item():.6f}")

In [13]:
def test(model, z_file, x_test, y_test, batch_size, n_class, DEVICE, epoch, mode):
  model.eval()
  total_count = len(x_test)
  correct_count = 0
  with torch.no_grad():
    while(len(x_test)) != 0:
      if len(x_test) < batch_size:
        batch_size = len(x_test)

      x_batch = np.zeros((batch_size, 128, 128, 3)) # 테스트 할때는 원본 이미지를 사용해야 함! 때문에 잘랐던 96,96 이 아닌거
      y_batch = np.zeros(batch_size) # 원핫 인코딩을 따로 안해줘도 됨! 이유는 나중에나옴

      for idx in range(batch_size):
        img = z_file.read(x_test.pop())
        img = np.frombuffer(img, np.int8)
        img = cv2.imdecode(img, cv2.IMREAD_COLOR)
        img = (img - np.mean(img, axis=(0, 1))) / np.std(img, axis = (0, 1))

        cls = y_test.pop()

        # train 데이터는 랜덤하게 했지만 테스트는 pop 해줌

        x_batch[idx] = img
        y_batch[idx] = cls
      

      x_batch = torch.FloatTensor(x_batch)
      y_batch = torch.FloatTensor(y_batch)
      x_batch = x_batch.permute(0, 3, 1, 2)
      x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)

      y_pred = model(x_batch)
      y_pred = torch.argmax(y_pred, dim = 1)
      correct_count += torch.sum(y_pred == y_batch) # y_pred와 y_batch를 직접 비교하여 같을때 T 다를때 F 로 반환(따라서 원핫 인코딩이 필요 없음)

  print(f"{mode} [{epoch}]  Accuracy : {correct_count / total_count *100}")
  f = open('accuracy.txt', 'a+')
  f.write(f"{mode} [{epoch}]  Accuracy : {correct_count / total_count *100}\n") # <- 줄바꿈 넣어야 함
  f.close()
  return

In [14]:
import torch.optim as optim

learning_rate = 0.01
batch_size = 32
max_epoch = 100
n_class = 10

# model = Simple_CNN()
model = Res_net_18()
model = model.to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9,
                      weight_decay=1e-4)
# SGD - > 확률적 경사 하강법

for i in range(1, max_epoch+1):
  train(model, optimizer, z_file, X_train, y_train, batch_size, n_class, DEVICE, i)
  test(model, z_file, X_train.copy(), y_train.copy(), batch_size, n_class, DEVICE, i, 'Train')
  test(model, z_file, X_test.copy(), y_test.copy(), batch_size, n_class, DEVICE, i, 'Test')

Train [1] [0]/336 Loss: 2.342348
Train [1] [100]/336 Loss: 2.115153
Train [1] [200]/336 Loss: 1.543580
Train [1] [300]/336 Loss: 1.132459
Train [1]  Accuracy : 46.433876037597656
Test [1]  Accuracy : 45.57949447631836
Train [2] [0]/336 Loss: 1.349220
Train [2] [100]/336 Loss: 1.282243
Train [2] [200]/336 Loss: 0.958692
Train [2] [300]/336 Loss: 1.201988
Train [2]  Accuracy : 57.60586929321289
Test [2]  Accuracy : 54.569095611572266
Train [3] [0]/336 Loss: 0.836492
Train [3] [100]/336 Loss: 1.543804
Train [3] [200]/336 Loss: 0.529468
Train [3] [300]/336 Loss: 0.616566
Train [3]  Accuracy : 65.6017837524414
Test [3]  Accuracy : 64.45022583007812
Train [4] [0]/336 Loss: 1.144271
Train [4] [100]/336 Loss: 1.179234
Train [4] [200]/336 Loss: 1.309636
Train [4] [300]/336 Loss: 0.773728
Train [4]  Accuracy : 66.77191162109375
Test [4]  Accuracy : 64.33877563476562
Train [5] [0]/336 Loss: 1.232022
Train [5] [100]/336 Loss: 1.082234
Train [5] [200]/336 Loss: 0.688936
Train [5] [300]/336 Loss: 0.

Resnet 34 모델

In [15]:
class Res_net_34(nn.Module):
  def __init__(self):
    super(Res_net_34, self).__init__()

    self.header = nn.Sequential(
        nn.Conv2d(3,64, kernel_size = 7, stride = 2, padding = 4),
        # nn.Conv2d(입력 채널, 출력 채널, kernel_size = 커널 사이즈, stride = 보폭)
        nn.BatchNorm2d(64),
        # nn.BatchNorm2d(입력 채널) -> ReLU를 쓸꺼라서 노멀리제이션 해줌
        nn.ReLU(),
        nn.MaxPool2d(3, stride = 2)
    )

    # Sequential 순차적 뉴럴네트워크 구성 실행
    self.block_1 = nn.Sequential(
        Bottle_Neck(64,64),
        Bottle_Neck(64,64),
        Bottle_Neck(64,64),
        nn.MaxPool2d(2, stride=2)
    )

    # 다음 블럭을 넘어갈때 인풋 채널과 아웃풋 채널이 달라 더할 수 없음.
    # Bottle_Neck 에 if is_downsample = True: 조건을 줌.

    self.block_2 = nn.Sequential(
        Bottle_Neck(64,128,is_downsample = True),
        Bottle_Neck(128,128),
        Bottle_Neck(128,128),
        Bottle_Neck(128,128),
        nn.MaxPool2d(2, stride=2)
    )

    self.block_3 = nn.Sequential(
        Bottle_Neck(128,256,is_downsample = True),
        Bottle_Neck(256,256),
        Bottle_Neck(256,256),
        Bottle_Neck(256,256),
        Bottle_Neck(256,256),
        Bottle_Neck(256,256),
        nn.MaxPool2d(2, stride=2)
    )

    self.block_4 = nn.Sequential(
        Bottle_Neck(256,512,is_downsample = True),
        Bottle_Neck(512,512),
        Bottle_Neck(512,512),
        nn.MaxPool2d(2, stride=2),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )

    self.linear = nn.Sequential(
        nn.Linear(512, 10)
    )

  def forward(self, input):
    x = self.header(input)
    x = self.block_1(x)
    x = self.block_2(x)
    x = self.block_3(x)
    x = self.block_4(x)
    # print(x.shape)
    x = F.avg_pool2d(x, kernel_size = x.shape[2]) # global averge pooling
    # 커널사이즈를 마지막 W와 같게 해줌 -> 결과적으로 (32,512,3,3) or (32,512,7,7) -> (32,512,1,1)로 바뀌고
    x = torch.flatten(x, start_dim = 1)
    # flatten 해주면 (32,512)의 모양이 됨.
    x = self.linear(x)
    return x

Resnet 50 모델

In [16]:
class Bottle_Neck_Two(nn.Module):
  def __init__(self, input_channel, hidden_channel, output_channel, is_downsample = False):
    super(Bottle_Neck, self).__init__()

    self.bottle_neck = nn.Sequential(
        nn.BatchNorm2d(input_channel),
        nn.ReLU(),
        nn.Conv2d(input_channel,hidden_channel,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(hidden_channel),
        nn.ReLU(),
        nn.Conv2d(hidden_channel,hidden_channel,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(hidden_channel),
        nn.ReLU(),
        nn.Conv2d(hidden_channel,hidden_channel,kernel_size=3,stride=1,padding=1),
        nn.BatchNorm2d(hidden_channel),
        nn.ReLU(),
        nn.Conv2d(hidden_channel,output_channel,kernel_size=3,stride=1,padding=1)
    )

    if is_downsample == True:  
      
      self.down_sample_conv = nn.Conv2d(input_channel, output_channel, kernel_size = 1,
                                       stride = 1, padding = 0)

  def forward(self, input):
    # indentity = input
    x = self.bottle_neck(input)
    if x.shape != input.shape:
      input = self.down_sample_conv(input) # -> 1X1 을 할땐 편향을 꺼주어야 한다?

    return x + input

In [17]:
class Res_net_50(nn.Module):
  def __init__(self):
    super(Res_net_50, self).__init__()
    self.header = nn.Sequential(
        nn.Conv2d(3,64, kernel_size = 7, stride = 2, padding = 4),
        # nn.Conv2d(입력 채널, 출력 채널, kernel_size = 커널 사이즈, stride = 보폭)
        nn.BatchNorm2d(64),
        # nn.BatchNorm2d(입력 채널) -> ReLU를 쓸꺼라서 노멀리제이션 해줌
        nn.ReLU(),
        nn.MaxPool2d(3, stride = 2)
    )

In [18]:
def train(model, optimizer, z_file, x_train, y_train, batch_size, n_class, DEVICE, epoch):
  model.train()
  for i in range(len(x_train)//batch_size):
    data, target = load_train_batch(z_file, batch_size, x_train, y_train, n_class)
    data, target = data.to(DEVICE), target.to(DEVICE) # 맨~ 처음에 디바이스를 GPU로 할지 CPU로 할지 정해뒀기 때문에 각 위치에 데이터를 보내는것
    optimizer.zero_grad()
    y_pred = model(data)
    criterion = nn.CrossEntropyLoss() 
    loss = criterion(y_pred, target) # -> CrossEntropyLoss 함수가 y_pred에 softmax를 적용 후 target과의 loss를 구해줌
    loss.backward()
    optimizer.step()
    if i%100==0:
      print(f"Train [{epoch}] [{i}]/{len(x_train)//batch_size} Loss: {loss.item():.6f}")

In [19]:
def test(model, z_file, x_test, y_test, batch_size, n_class, DEVICE, epoch, mode):
  model.eval()
  total_count = len(x_test)
  correct_count = 0
  with torch.no_grad():
    while(len(x_test)) != 0:
      if len(x_test) < batch_size:
        batch_size = len(x_test)

      x_batch = np.zeros((batch_size, 128, 128, 3)) # 테스트 할때는 원본 이미지를 사용해야 함! 때문에 잘랐던 96,96 이 아닌거
      y_batch = np.zeros(batch_size) # 원핫 인코딩을 따로 안해줘도 됨! 이유는 나중에나옴

      for idx in range(batch_size):
        img = z_file.read(x_test.pop())
        img = np.frombuffer(img, np.int8)
        img = cv2.imdecode(img, cv2.IMREAD_COLOR)
        img = (img - np.mean(img, axis=(0, 1))) / np.std(img, axis = (0, 1))

        cls = y_test.pop()

        # train 데이터는 랜덤하게 했지만 테스트는 pop 해줌

        x_batch[idx] = img
        y_batch[idx] = cls
      

      x_batch = torch.FloatTensor(x_batch)
      y_batch = torch.FloatTensor(y_batch)
      x_batch = x_batch.permute(0, 3, 1, 2)
      x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)

      y_pred = model(x_batch)
      y_pred = torch.argmax(y_pred, dim = 1)
      correct_count += torch.sum(y_pred == y_batch) # y_pred와 y_batch를 직접 비교하여 같을때 T 다를때 F 로 반환(따라서 원핫 인코딩이 필요 없음)

  print(f"{mode} [{epoch}]  Accuracy : {correct_count / total_count *100}")
  f = open('accuracy.txt', 'a+')
  f.write(f"{mode} [{epoch}]  Accuracy : {correct_count / total_count *100}\n") # <- 줄바꿈 넣어야 함
  f.close()
  return

In [ ]:
import torch.optim as optim

learning_rate = 0.01
batch_size = 32
max_epoch = 100
n_class = 10

# model = Simple_CNN()
# model = Res_net_18()
model = Res_net_34()
model = model.to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum = 0.9,
                      weight_decay=1e-4)
# SGD - > 확률적 경사 하강법

for i in range(1, max_epoch+1):
  train(model, optimizer, z_file, X_train, y_train, batch_size, n_class, DEVICE, i)
  test(model, z_file, X_train.copy(), y_train.copy(), batch_size, n_class, DEVICE, i, 'Train')
  test(model, z_file, X_test.copy(), y_test.copy(), batch_size, n_class, DEVICE, i, 'Test')

Train [1] [0]/336 Loss: 2.342233
Train [1] [100]/336 Loss: 1.733286
Train [1] [200]/336 Loss: 1.626455
Train [1] [300]/336 Loss: 1.819630
Train [1]  Accuracy : 41.103267669677734
Test [1]  Accuracy : 41.15898895263672
Train [2] [0]/336 Loss: 1.607325
Train [2] [100]/336 Loss: 1.793281
Train [2] [200]/336 Loss: 1.029169
Train [2] [300]/336 Loss: 1.353619
Train [2]  Accuracy : 49.498512268066406
Test [2]  Accuracy : 48.142642974853516
Train [3] [0]/336 Loss: 1.190884
Train [3] [100]/336 Loss: 0.979290
Train [3] [200]/336 Loss: 1.188309
Train [3] [300]/336 Loss: 0.826943
Train [3]  Accuracy : 61.03268814086914
Test [3]  Accuracy : 59.175331115722656
Train [4] [0]/336 Loss: 0.934735
Train [4] [100]/336 Loss: 1.319257
Train [4] [200]/336 Loss: 1.271620
Train [4] [300]/336 Loss: 0.899093
Train [4]  Accuracy : 67.12481689453125
Test [4]  Accuracy : 66.60475158691406
Train [5] [0]/336 Loss: 0.832174
Train [5] [100]/336 Loss: 0.538747
Train [5] [200]/336 Loss: 0.915808
Train [5] [300]/336 Loss:

오버피팅을 해결하기위한 전략(데이터 증강)
1. 이미지 좌우 반전

opencv 라이브러리 사용

In [ ]:
import cv2
def test(model, z_file, x_test, y_test, batch_size, n_class, DEVICE, epoch, mode):
  model.eval()
  total_count = len(x_test)
  correct_count = 0
  with torch.no_grad():
    while(len(x_test)) != 0:
      if len(x_test) < batch_size:
        batch_size = len(x_test)

      x_batch = np.zeros((batch_size, 128, 128, 3)) # 테스트 할때는 원본 이미지를 사용해야 함! 때문에 잘랐던 96,96 이 아닌거
      y_batch = np.zeros(batch_size) # 원핫 인코딩을 따로 안해줘도 됨! 이유는 나중에나옴

      for idx in range(batch_size):
        img = z_file.read(x_test.pop())
        img = np.frombuffer(img, np.int8)
        img = cv2.imdecode(img, cv2.IMREAD_COLOR)
        img = (img - np.mean(img, axis=(0, 1))) / np.std(img, axis = (0, 1))

        cls = y_test.pop()

        # train 데이터는 랜덤하게 했지만 테스트는 pop 해줌

        x_batch[idx] = img
        y_batch[idx] = cls
      

      x_batch = torch.FloatTensor(x_batch)
      y_batch = torch.FloatTensor(y_batch)
      x_batch = x_batch.permute(0, 3, 1, 2)
      x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)

      y_pred = model(x_batch)
      y_pred = torch.argmax(y_pred, dim = 1)
      correct_count += torch.sum(y_pred == y_batch) # y_pred와 y_batch를 직접 비교하여 같을때 T 다를때 F 로 반환(따라서 원핫 인코딩이 필요 없음)

  print(f"{mode} [{epoch}]  Accuracy : {correct_count / total_count *100}")
  f = open('accuracy.txt', 'a+')
  f.write(f"{mode} [{epoch}]  Accuracy : {correct_count / total_count *100}\n") # <- 줄바꿈 넣어야 함
  f.close()
  return

In [ ]:
class Res_net_34(nn.Module):
  def __init__(self):
    super(Res_net_34, self).__init__()

    self.header = nn.Sequential(
        nn.Conv2d(3,64, kernel_size = 7, stride = 2, padding = 4),
        # nn.Conv2d(입력 채널, 출력 채널, kernel_size = 커널 사이즈, stride = 보폭)
        nn.BatchNorm2d(64),
        # nn.BatchNorm2d(입력 채널) -> ReLU를 쓸꺼라서 노멀리제이션 해줌
        nn.ReLU(),
        nn.MaxPool2d(3, stride = 2)
    )

    # Sequential 순차적 뉴럴네트워크 구성 실행
    self.block_1 = nn.Sequential(
        Bottle_Neck(64,64),
        Bottle_Neck(64,64),
        Bottle_Neck(64,64),
        nn.MaxPool2d(2, stride=2)
    )

    # 다음 블럭을 넘어갈때 인풋 채널과 아웃풋 채널이 달라 더할 수 없음.
    # Bottle_Neck 에 if is_downsample = True: 조건을 줌.

    self.block_2 = nn.Sequential(
        Bottle_Neck(64,128,is_downsample = True),
        Bottle_Neck(128,128),
        Bottle_Neck(128,128),
        Bottle_Neck(128,128),
        nn.MaxPool2d(2, stride=2)
    )

    self.block_3 = nn.Sequential(
        Bottle_Neck(128,256,is_downsample = True),
        Bottle_Neck(256,256),
        Bottle_Neck(256,256),
        Bottle_Neck(256,256),
        Bottle_Neck(256,256),
        Bottle_Neck(256,256),
        nn.MaxPool2d(2, stride=2)
    )

    self.block_4 = nn.Sequential(
        Bottle_Neck(256,512,is_downsample = True),
        Bottle_Neck(512,512),
        Bottle_Neck(512,512),
        nn.MaxPool2d(2, stride=2),
        nn.BatchNorm2d(512),
        nn.ReLU()
    )

    self.linear = nn.Sequential(
        nn.Linear(512, 10)
    )

  def forward(self, input):
    x = self.header(input)
    x = self.block_1(x)
    x = self.block_2(x)
    x = self.block_3(x)
    x = self.block_4(x)
    # print(x.shape)
    x = F.avg_pool2d(x, kernel_size = x.shape[2]) # global averge pooling
    # 커널사이즈를 마지막 W와 같게 해줌 -> 결과적으로 (32,512,3,3) or (32,512,7,7) -> (32,512,1,1)로 바뀌고
    x = torch.flatten(x, start_dim = 1)
    x = torch.nn.Dropout(p=0.5, inplace=False)
    # flatten 해주면 (32,512)의 모양이 됨.
    x = self.linear(x)
    return x